In [ ]:
# Imports
import os
import sys
import math
import networkx as nx

# Ensure notebook is running from Tournesol-Stats dir
_pwd = os.path.realpath('.').split(os.sep)
if 'src' in _pwd:
	while _pwd[-1] != 'src':
		_pwd.pop()
	_pwd.pop() # Go up from src dir to Tournesol-Stats
	os.chdir(os.sep.join(_pwd))
print(os.path.realpath('.'))

# Local project requirements
sys.path.append('src/py')
from model.tournesol_api import TournesolAPI, get

In [ ]:
# PARAMETERS & Loading Dataset
TOURNESOL_API=TournesolAPI(input('JWT (example: "Bearer xxxxxxxxx")'))
TOURNESOL_API.loadCache(f"./data/Tournesol_API_cache-{TOURNESOL_API.username}.json.gz")

# Load dataset
TOURNESOL_API.getMyComparedVideos()
comparisons = TOURNESOL_API.getAllMyComparisons()

In [ ]:
# Prepare graphs

public_undgraph = nx.Graph()
private_digraph = nx.DiGraph()
for cdata in comparisons:
	if cdata.get('is_public', False):
		public_undgraph.add_edge(cdata['entity_a'], cdata['entity_b'])

	score = [dta['score'] for dta in cdata['criteria_scores'] if dta['criteria'] == 'largely_recommended'][0]
	if score >= 0:
		private_digraph.add_edge(cdata['entity_a'], cdata['entity_b'])
	if score <= 0:
		private_digraph.add_edge(cdata['entity_b'], cdata['entity_a'])
private_undgraph = private_digraph.to_undirected(as_view=True)

videos = {vid: TOURNESOL_API.getVData(vid, useCache=True, saveCache=False) for vid in private_undgraph.nodes}
for vid,video in videos.items():
	if get(video, False, 'individual_rating', 'is_public') and (vid not in public_undgraph):
		public_undgraph.add_node(vid)

def get_obj(cntrs):
	if cntrs <= 2:
		return 2
	fact = math.floor(math.log10(cntrs))
	return (9*fact) + cntrs//(10**fact)
def nb_needed_cmps(vid):
	cntrs = videos[vid]['collective_rating']['n_contributors']	
	cmps = videos[vid]['individual_rating']['n_comparisons']
	return get_obj(cntrs) - cmps

_cmps_needed = {vid: nb_needed_cmps(vid) for vid in videos if vid in public_undgraph}
need_more_cmps = {vid: need for vid,need in _cmps_needed.items() if need > 0}
print('Videos', len(videos))
print('Comparisons', len(comparisons))
print('Public', public_undgraph)
print('Private', private_undgraph)
print('Directed', private_digraph)

In [ ]:
sorted_needed = list(need_more_cmps)
def resort():
	global sorted_needed
	sorted_needed.sort(key=lambda v: (-public_undgraph.degree[v], need_more_cmps[v], videos[v]['collective_rating']['tournesol_score']), reverse=True)
resort()

print(' vid           public private need')
for v in sorted_needed:
	print(v, f"{public_undgraph.degree[v]:5d} +{private_undgraph.degree[v]-public_undgraph.degree[v]:6d} :{need_more_cmps[v]:+4d}")

In [ ]:
# Find all public videos having <2 public comparisons
suggested = 0
def suggest(filter_degree):
	global suggested
	for vid in sorted(
			(v for v in public_undgraph.nodes if public_undgraph.degree[v] == filter_degree),
			key=lambda v: (-videos[v]['collective_rating']['n_contributors'], private_undgraph.degree[v])
		):

		# Find the video in sorted_needed the furthest away from vid in private graph (if multiple matches, keep the first matched one)
		# and that is not yet reachable from or to vid
		local = set(nx.single_source_shortest_path(private_digraph, vid)).union(set(nx.single_target_shortest_path(private_digraph, vid)))
		distances = nx.single_source_shortest_path_length(private_undgraph, vid, cutoff=99)
		best_c = None
		for candidate in sorted_needed:
			if candidate in local:
				continue
			if candidate not in distances:
				best_c = candidate
				break
			if (best_c is None) or (distances[best_c] < distances[candidate]):
				best_c = candidate

		if not best_c:
			continue

		# Print comparison
		suggested += 1
		print(f"{suggested:4d}: https://tournesol.app/comparison?uidA={vid}&uidB={best_c} (dist={distances.get(best_c,' ~')})")
		# Update the graphs & data accounting for this comparison
		public_undgraph.add_edge(vid, best_c)
		private_digraph.add_edge(vid, best_c)
		private_digraph.add_edge(best_c, vid)
		if vid in need_more_cmps:
			need_more_cmps[vid] -= 1
			if need_more_cmps[vid] <= 0:
				need_more_cmps.pop(vid)
				sorted_needed.remove(vid)
		need_more_cmps[best_c] -= 1
		if need_more_cmps[best_c] <= 0:
			need_more_cmps.pop(best_c)
			sorted_needed.remove(best_c)
		resort()

print('\n------------\n     NO PUBLIC COMPARISONS\n')
suggest(0)
if suggested < 50:
	print('\n------------\n     SINGLE PUBLIC COMPARISON\n')
	suggest(1)
if suggested < 50:
	print('\n------------\n     ONLY 2 PUBLIC COMPARISONS\n')
	suggest(2)
